In [201]:
import getpass
import arcgis.features as features
import arcgis.geometry as geometry
from arcgis.gis import GIS

password = getpass.getpass("Please enter password: ")
gis = GIS('https://browns.maps.arcgis.com', 'johnnyDev', password)
print("Successfully logged in to {} as {}".format(gis.properties.urlKey + '.' + gis.properties.customBaseUrl,
                                                 gis.users.me.username))

Please enter password: ········
Successfully logged in to browns.maps.arcgis.com as johnnyDev


In [204]:
# search for the item that you previusly made with the Create Layer tool
feature_item = gis.content.search(query="""title: "Griffith Park*" """, 
                                  item_type="Feature Service")
# the search returns a list of items, so retrieve the item
griffith_item = feature_item[0]
# retrive the Feature Layer item with layers property of the item
griffith_lyr = griffith_item.layers[0]
# print values to ensure you have the layer
print("{} is a {}".format(griffith_lyr.properties.name, griffith_lyr.properties.type))

griffith_park_mac is a Feature Layer


In [205]:
# load csv file with coordinate values and attributes for new features into a pandas dataframe
import pandas
feat_records = pandas.read_csv(r'C:\Data\CSV\la_area_values.csv')
feat_records

,Name,Type,Surface,Latitude,Longitude
0,Griffith Park,Entrance,Concrete,34.136555,-118.294197
1,Hollywood Sign,Attraction,Mountainous,34.134021,-118.323865
2,Paramount Pictures,Attraction,Concrete,34.085423,-118.321341


In [206]:
# create a variable for the spatial reference
sr = {'wkid': 4236}

# construct points and attributes lists from the dataframe
pts = []
atts = []
for index, row in feat_records.iterrows():
    pt = geometry.Point({'x': row['Longitude'],
                    'y': row['Latitude'],
                    'spatialReference': sr
                   })
    att = {'name' : row['Name'],
          'type': row['Type'],
          'surface': row['Surface'],
          'OBJECTID': index + 1}
    atts.append(att)
    pts.append(pt)

In [208]:
atts

[{'OBJECTID': 1,
  'name': 'Griffith Park',
  'surface': ' Concrete',
  'type': ' Entrance'},
 {'OBJECTID': 2,
  'name': 'Hollywood Sign',
  'surface': ' Mountainous',
  'type': ' Attraction'},
 {'OBJECTID': 3,
  'name': 'Paramount Pictures',
  'surface': ' Concrete',
  'type': ' Attraction'}]

In [214]:
feats = []
for i in range(0, 3):
    feat = features.Feature(geometry=pts[i], attributes=atts[i])
    feats.append(feat)

In [215]:
for feat in feats:
    print(feat.get_value('name'))

Griffith Park
Hollywood Sign
Paramount Pictures


In [ ]:
griffith_lyr.edit_features(adds=feats) # feats should be a list? doc says 'array'? np array?

In [200]:
map = gis.map('Los Angeles', 10)
map

In [199]:
#map.add_layer(griffith_lyr)